In [7]:
!pip uninstall typing_extensions -y
!pip install --upgrade typing_extensions


!pip install torch torchvision


Found existing installation: typing_extensions 4.14.0
Uninstalling typing_extensions-4.14.0:
  Successfully uninstalled typing_extensions-4.14.0
  Using cached typing_extensions-4.14.0-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.14.0-py3-none-any.whl (43 kB)


In [8]:
import torch
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
from torchvision.utils import make_grid
import torchvision.transforms as T
from scipy.ndimage import zoom
from PIL import Image, ImageDraw, ImageFont

# Función para cargar y obtener cortes de una imagen NIfTI
def load_nifti_image(nifti_path):
    img = nib.load(nifti_path).get_fdata()
    return img

def get_slices(img):
    """Obtiene cortes Axial, Coronal y Sagital y los rota 90° antihorario."""
    # Verificar las dimensiones de la imagen
    if len(img.shape) == 4:  # Si es 4D, tomar el primer volumen
        img = img[..., 0]
    elif len(img.shape) != 3:  # Si no es 3D, lanzar un error
        raise ValueError(f"Se esperaba una imagen 3D, pero se obtuvo una con forma {img.shape}")

    # Extraer las dimensiones
    z, y, x = img.shape

    # Obtener los cortes
    axial = np.rot90(img[z // 2, :, :])      # Vista axial
    coronal = np.rot90(img[:, y // 2, :])    # Vista coronal
    sagittal = np.rot90(img[:, :, x // 2])   # Vista sagital

    return axial, coronal, sagittal

def normalize_image(image):
    """ Normaliza la imagen entre 0 y 1 """
    return (image - np.min(image)) / (np.max(image) - np.min(image) + 1e-8)  # Evita división por cero

from torchvision import transforms

def resize_image(image, final_size=(256, 256)):
    """
    Redimensiona la imagen preservando la relación de aspecto y luego aplica padding para alcanzar el tamaño final.
    """
    pil_img = T.ToPILImage()(image)

    # Obtener el tamaño original
    w, h = pil_img.size  

    # Calcular la escala manteniendo la proporción
    scale = min(final_size[0] / w, final_size[1] / h)  
    new_w, new_h = int(w * scale), int(h * scale)  

    # Redimensionar con la escala correcta
    resized = pil_img.resize((new_w, new_h), Image.BILINEAR)

    # Calcular padding necesario
    pad_left = (final_size[0] - new_w) // 2
    pad_top = (final_size[1] - new_h) // 2
    pad_right = final_size[0] - new_w - pad_left
    pad_bottom = final_size[1] - new_h - pad_top

    # Aplicar padding
    transform_pad = transforms.Pad((pad_left, pad_top, pad_right, pad_bottom), fill=0)
    padded = transform_pad(T.ToTensor()(resized))

    return padded.numpy().squeeze()  # Convertimos de tensor a array




def nifti_to_tensor(nifti_path, target_size=(256, 256)):
    """
    Carga un archivo NIfTI y lo convierte en un tensor normalizado,
    asegurando que todas las imágenes tengan el mismo tamaño.
    """
    img = load_nifti_image(nifti_path)
    axial, coronal, sagittal = get_slices(img)

    # Normalizar imágenes
    axial = normalize_image(axial)
    coronal = normalize_image(coronal)
    sagittal = normalize_image(sagittal)

    # Redimensionar todas las imágenes al mismo tamaño
    axial = resize_image(axial, target_size)
    coronal = resize_image(coronal, target_size)
    sagittal = resize_image(sagittal, target_size)

    # Convertir a tensores de PyTorch (1 canal)
    transform = T.Compose([T.ToTensor(), T.Normalize(mean=[0.5], std=[0.5])])
    axial_t = transform(axial).unsqueeze(0)
    coronal_t = transform(coronal).unsqueeze(0)
    sagittal_t = transform(sagittal).unsqueeze(0)

    return torch.cat([axial_t, coronal_t, sagittal_t], dim=0)

def add_labels(image, subject_id):
    """Agrega etiquetas 'Antes' y 'Después' a la izquierda y un título arriba."""
    draw = ImageDraw.Draw(image)
    
    try:
        font = ImageFont.truetype("arial.ttf", 30)  # Intenta usar Arial
    except:
        font = ImageFont.load_default()  # Fuente por defecto si Arial no está disponible

    W, H = image.size
    margin = 40  # Espacio para etiquetas

    # Crear una imagen más grande para incluir etiquetas
    new_image = Image.new("RGB", (W, H + margin * 2), (255, 255, 255))
    new_image.paste(image, (0, margin))

    draw = ImageDraw.Draw(new_image)

    # Agregar etiquetas 'Antes' y 'Después'
    draw.text((10, margin + H // 4), "ANTES", fill="red", font=font)
    draw.text((10, margin + (3 * H) // 4), "DESPUES", fill="red", font=font)

    # Agregar título centrado arriba
    title = f"{subject_id} DeepBrainNet Preprocessing Pipeline"
    text_width = draw.textbbox((0, 0), title, font=font)[2]  # Extrae solo el ancho del texto
    draw.text(((W - text_width) // 2, 5), title, fill="black", font=font)

    return new_image

def plot_brain_grid(subject_id, before_path, after_path, output_dir):
    """Genera y guarda la imagen comparativa antes y después del preprocesamiento."""
    brain_slices = [nifti_to_tensor(p) for p in [before_path, after_path]]
    grid = make_grid(torch.cat(brain_slices, dim=0), nrow=3, padding=2, normalize=True)
    grid_np = grid.numpy().transpose(1, 2, 0)

    # Convertir a imagen de PIL y agregar etiquetas
    img = Image.fromarray((grid_np * 255).astype(np.uint8))
    labeled_img = add_labels(img, subject_id)

    # Guardar la imagen
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"{subject_id}.jpg")
    labeled_img.save(output_path)

# Directorios de entrada y salida
input_dir_before = "/data/Lautaro/Documentos/BrainAgeCOVID/DeepBrainNet/T1_cropped"
input_dir_after = "/data/Lautaro/Documentos/BrainAgeCOVID/ADNI/ADNI_3_T1/brainagenext/preprocesedCP"
output_dir = "./Preprocessing_analysis"

# Obtener la lista de sujetos a procesar basado en el nombre del archivo (sin extensión) de la carpeta "before"
subject_ids = [os.path.splitext(os.path.splitext(f)[0])[0] 
               for f in os.listdir(input_dir_before) if f.endswith(".nii.gz")]

# Generar imágenes para cada sujeto
for subject_id in subject_ids:
    before_path = os.path.join(input_dir_before, f"{subject_id}.nii.gz")
    after_path = os.path.join(input_dir_after, f"{subject_id}.nii.gz")  # ahora en after, los archivos terminan en .nii.gz

    if os.path.exists(before_path) and os.path.exists(after_path):
        plot_brain_grid(subject_id, before_path, after_path, output_dir)
        print(f"Imagen guardada: {subject_id}.jpg")
    else:
        print(f"Archivos no encontrados para {subject_id}: {before_path} o {after_path}")


Imagen guardada: CP0001.jpg
Imagen guardada: CP0002.jpg
Imagen guardada: CP0003.jpg
Imagen guardada: CP0004.jpg
Imagen guardada: CP0006.jpg
Imagen guardada: CP0007.jpg
Imagen guardada: CP0008.jpg
Imagen guardada: CP0009.jpg
Imagen guardada: CP0010.jpg
Imagen guardada: CP0011.jpg
Imagen guardada: CP0013.jpg
Imagen guardada: CP0014.jpg
Imagen guardada: CP0019.jpg
Imagen guardada: CP0020.jpg
Imagen guardada: CP0022.jpg
Imagen guardada: CP0023.jpg
Imagen guardada: CP0024.jpg
Imagen guardada: CP0027.jpg
Imagen guardada: CP0028.jpg
Imagen guardada: CP0029.jpg
Imagen guardada: CP0030.jpg
Imagen guardada: CP0031.jpg
Imagen guardada: CP0032.jpg
Imagen guardada: CP0033.jpg
Imagen guardada: CP0035.jpg
Imagen guardada: CP0036.jpg
Imagen guardada: CP0037.jpg
Imagen guardada: CP0038.jpg
Imagen guardada: CP0039.jpg
Imagen guardada: CP0040.jpg
Imagen guardada: CP0041.jpg
Imagen guardada: CP0042.jpg
Imagen guardada: CP0043.jpg
Imagen guardada: CP0044.jpg
Imagen guardada: CP0071.jpg
Imagen guardada: CP0

In [5]:
# Directorios de entrada y salida
input_dir_before = "/data/Lautaro/Documentos/BrainAgeCOVID/ADNI/ADNI_3_T1/sampled_imgs"
input_dir_after = "/data/Lautaro/Documentos/BrainAgeCOVID/ADNI/ADNI_3_T1/freesurfer+fsl2"
output_dir = "./Preprocessing_analysis"

# Obtener lista de sujetos a procesar (usar el nombre completo del archivo sin la extensión)
subject_ids = [os.path.splitext(f)[0] for f in os.listdir(input_dir_before) if f.endswith(".nii")]

# Generar imágenes para cada sujeto
for subject_id in subject_ids:
    before_path = os.path.join(input_dir_before, f"{subject_id}.nii")
    after_path = os.path.join(input_dir_after, f"{subject_id}.nii", "mri", "cropped.nii.gz")

    if os.path.exists(before_path) and os.path.exists(after_path):
        plot_brain_grid(subject_id, before_path, after_path, output_dir)
        print(f"Imagen guardada: {subject_id}.jpg")
    else:
        print(f"Archivos no encontrados para {subject_id}")

ValueError: too many values to unpack (expected 3)

In [20]:
import nibabel as nib

# Ruta al archivo NIfTI
ruta_imagen = '/data/Lautaro/Documentos/BrainAgeCOVID/ADNI/ADNI_3_T1/IXI/freesurfer+fsl(IXI)/IXI013-HH-1212-T1/mri/cropped.nii.gz'  # Modifica esta ruta

# Cargar la imagen
img = nib.load(ruta_imagen)

# Obtener el array de datos
data = img.get_fdata()

# Mostrar el tamaño (dimensiones) de la imagen
print("Tamaño de la imagen NIfTI:", data.shape)

Tamaño de la imagen NIfTI: (167, 212, 160)
